In [7]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

In [8]:
df = pd.read_csv("dataset.csv")  # Adjust name if needed
print(df.columns)
print(df.head())

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   

In [9]:
base_price = 10
alpha = 5
λ = 0.5

vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
traffic_weights = {'low': 0.5, 'medium': 1.0, 'high': 1.5}

In [10]:
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_weights)

In [11]:
df['BaselinePrice'] = base_price

for lot in df['SystemCodeNumber'].unique():
    lot_df = df[df['SystemCodeNumber'] == lot].sort_values(['LastUpdatedDate', 'LastUpdatedTime'])
    prices = [base_price]

    for i in range(1, len(lot_df)):
        prev_occ = lot_df.iloc[i - 1]['Occupancy']
        capacity = lot_df.iloc[i - 1]['Capacity']
        occ_ratio = prev_occ / capacity if capacity else 0
        next_price = prices[-1] + alpha * occ_ratio
        prices.append(next_price)

    df.loc[lot_df.index, 'BaselinePrice'] = prices

/tmp/ipython-input-11-987498443.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10, np.float64(10.433275563258233), np.float64(10.901213171577124), np.float64(11.499133448873485), np.float64(12.287694974003468), np.float64(13.457538994800695), np.float64(14.870017331022531), np.float64(16.47313691507799), np.float64(18.26689774696707), np.float64(20.337954939341422), np.float64(22.452339688041597), np.float64(24.558058925476605), np.float64(26.646447140381284), np.float64(28.630849220103986), np.float64(30.35528596187175), np.float64(31.915077989601386), np.float64(33.36221837088388), np.float64(34.68804159445407), np.float64(35.84055459272097), np.float64(36.031195840554595), np.float64(36.22183708838822), np.float64(36.49046793760832), np.float64(36.89774696707106), np.float64(37.712305025996535), np.float64(38.81282495667244), np.float64(40.13864818024263), np.float64(41.646447140381284), np.flo

In [12]:
df['RawDemand'] = (
    1.0 * (df['Occupancy'] / df['Capacity']) +
    1.0 * df['QueueLength'] -
    1.0 * df['TrafficLevel'] +
    1.0 * df['IsSpecialDay'] +
    1.0 * df['VehicleWeight']
)

df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
)

df['DemandPrice'] = base_price * (1 + λ * df['NormalizedDemand'])
df['DemandPrice'] = df['DemandPrice'].clip(lower=0.5 * base_price, upper=2.0 * base_price)

In [13]:
sample_lot = df['SystemCodeNumber'].unique()[0]
plot_df = df[df['SystemCodeNumber'] == sample_lot].sort_values(['LastUpdatedDate', 'LastUpdatedTime']).reset_index(drop=True)

p = figure(title=f"Pricing for Lot {sample_lot}", x_axis_label="Time Index", y_axis_label="Price")
p.line(plot_df.index, plot_df['BaselinePrice'], legend_label="Baseline Price", color="blue", line_width=2)
p.line(plot_df.index, plot_df['DemandPrice'], legend_label="Demand-Based Price", color="green", line_width=2)
p.legend.location = "top_left"
show(p)